In [1]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import max_error
import numpy as np

In [2]:
data = pd.read_pickle(r"../data/B00020S.pkl")
data['Date'] = pd.to_datetime(data['Date'])

NameError: name 'data' is not defined

# TODO
 - **DONE** znaleźć prawidłowe offset_values — Max
 - **DONE** wykresy błędów — Adam
 - **DONE** nałożyć wartość bezwzględną na histogram błędu bezwzględnego — Adam
 - **DONE** upiększyć wykresy — Max
 - dodać do temp kolumnę z datami — Max
 - może jakiś opis matematyczny w markdownie tej regresji czy coś — Max albo Adam
 - **DONE** zautomatyzować dobór offset_values - Max

In [3]:
# Słownik ze stacjami i odpowiadającymi im przesunięciom
stations_offset_values_dict = {
    'GŁOGÓW': 0,
    'ŚCINAWA': 1,
    'MALCZYCE': 1,
    'BRZEG DOLNY': 1,
    'OŁAWA': 2,
    'BRZEG': 2,
    'RACIBÓRZ-MIEDONIA': 3,
    'KRZYŻANOWICE': 3,
    'OLZA': 3,
    'CHAŁUPKI': 3
}

# Stacje
stations = ['GŁOGÓW', 'ŚCINAWA', 'MALCZYCE', 'BRZEG DOLNY', 'OŁAWA', 'BRZEG', 'RACIBÓRZ-MIEDONIA', 'KRZYŻANOWICE', 'OLZA', 'CHAŁUPKI']

# Lista różnic w dniach między Głogowem a kolejną stacją
offset_values = [stations_offset_values_dict[station] for station in stations]

# Rok, od którego chcemy trenować i testować model
start_year = 2019

# Grupowanie po dniach i stacjach
data_grouped = data.groupby(['Date', 'Station'])['B00020S'].mean().reset_index()

# Osobny dataframe dla każdej stacji i tylko rekordy od danego roku
station_datas = [data_grouped[(data_grouped['Station'] == station) & (data_grouped['Date'].dt.year >= start_year)].reset_index() for station in stations]

# Połączenie kolumn z poziomem wody z każdej stacji w jeden dataframe
temp = pd.concat([station_data['B00020S'] for station_data in station_datas], axis='columns').reset_index(drop=True)

# Zmiana nazw kolumn
temp.columns = stations

# Przesunięcie każdej kolumny o odpowiednią liczbę dni
for i, col in enumerate(temp.columns):
    temp[col] = temp[col].shift(periods=-offset_values[i])

# Usunięcie NA, możliwe, że później można to dopracować
temp = temp.dropna()

# Zmienne służące do podziału zbioru danych na zbiór treningowy i zbiór testujący
test_proportion = 0.2
train_proportion = 1 - test_proportion
split_point = int(len(temp) * train_proportion)

# Podział na zbiór treningowy i zbiór testowy
train_data, test_data = temp.iloc[:split_point], temp.iloc[split_point:]

# Zmienne niezależne
x = train_data.iloc[:, 1:]
x = sm.add_constant(x)

# Zmienna niezależna
y = train_data.iloc[:, 0]

# Dopasowanie modelu
model = sm.OLS(y, x).fit()

# Podsumowanie
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 GŁOGÓW   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     2095.
Date:                Wed, 20 Dec 2023   Prob (F-statistic):               0.00
Time:                        21:02:32   Log-Likelihood:                -6066.1
No. Observations:                1383   AIC:                         1.215e+04
Df Residuals:                    1373   BIC:                         1.220e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               140.0133      4.45

## Resztki modelu

In [ ]:
# Dataframe z resztami modelu
residuals = pd.DataFrame(model.resid)

# Wykres reszt
fig, axes = plt.subplots(figsize=(36,32), ncols=1, nrows=2, gridspec_kw={'hspace': 0.3})
residuals.plot(ax = axes[0], linewidth=3)
axes[0].set_xlim(0, len(residuals))
axes[0].set_title('Reszty', fontsize=30)
axes[0].legend().set_visible(False)
axes[0].grid()

# Wykres gęstości jądra reszt / rozkład prawdopodobieństwa reszt
residuals.plot(kind='kde', ax=axes[1], linewidth=4)
axes[1].set_xlim(-100, 100)
axes[1].set_ylim(0, 0.03)
axes[1].set_title('Wykres gęstości jądra / rozkładu prawdopodobieństwa reszt', fontsize=30)
axes[1].set_ylabel('Gęstość', fontsize=20)
axes[1].legend().set_visible(False)
axes[1].grid()

plt.show()

# Statystyki opisowe reszt
print(residuals.describe())

## Predykcja modelu

In [ ]:
# Przewidywanie wartości na podstawie modelu
model.predict(x)

# To samo co wyżej, ale ręcznie
test_predict = model.params['const'] + sum(model.params[param] * test_data[param] for param in model.params.index[1:])

## Metryki błędu

In [ ]:
# Prawdziwe dane testowe
y_true = test_data.iloc[:, 0]

# Błąd średniokwadratowy
mse = mean_squared_error(y_true=y_true, y_pred=test_predict, squared=True)

# Pierwiastek z błędu kwadratowego
rmse = mean_squared_error(y_true=y_true, y_pred=test_predict, squared=False)

# Błąd procentowy średniokwadratowy
mape = mean_absolute_percentage_error(y_true=y_true, y_pred=test_predict)

# Maksymalny błąd bezwzględny
max_absolute_error = max_error(y_true=y_true, y_pred=test_predict)

# Maksymalny błąd względny
max_relative_error = max(abs((train_data['GŁOGÓW'] - test_predict) / test_data['GŁOGÓW']))

# Bezwzględny błąd treningowy
train_absolute_error = abs(train_data['GŁOGÓW'] - test_predict)

# Względny błąd treningowy
train_relative_error = abs((train_data['GŁOGÓW'] - model.predict()) / train_data['GŁOGÓW'])

# Bezwzględny błąd testowy
test_absolute_error = abs(test_data['GŁOGÓW'] - test_predict)

# Względny błąd testowy
test_relative_error = abs((test_data['GŁOGÓW'] - test_predict) / test_data['GŁOGÓW'])

# Wyświetlenie niektórych powyższych metryk
print('Test MSE: %.3f' % mse)
print('Test RMSE: %.3f' % rmse)
print('Test MAPE: %.3f' % mape)
print('Test max error: %.3f' % max_absolute_error)
print('Test max relative error: %.5f' % max_relative_error)

## Wykresy predykcji, błędu bezwzględnego i błędu względnego

In [ ]:
# Tymczasowa oś X, trzeba będzie zmienić na prawdziwą datę
date_train = np.arange(0, len(train_data))
date_test = np.arange(len(train_data), len(train_data) + len(test_data))

# Okno wykresów
fig, axes = plt.subplots(figsize=(36, 32), nrows=3, ncols=1, gridspec_kw={'hspace': 0.3})

# Wykres predykcji
axes[0].plot(date_train, train_data['GŁOGÓW'], color='b', label='Faktyczne dane', linewidth=3)
axes[0].plot(date_train, model.predict(), 'r', label='Model', linewidth=3)
axes[0].plot(date_test, test_data['GŁOGÓW'], color='r', label='Faktyczne dane (test)', linewidth=3)
axes[0].plot(date_test, test_predict, color='m', label='Model (test)', linewidth=3)
axes[0].set_xlabel('Data', fontsize=15)
axes[0].set_ylabel('Poziom wody (cm)', fontsize=15)
axes[0].set_title('Predykcja poziomu wody w Głogowie', fontsize=30)
axes[0].legend(loc='upper right', fontsize=20)
axes[0].grid()

# Wykres błędu bezwzględnego
axes[1].plot(date_train, train_absolute_error, color='b', label='Błąd bezwzględny', linewidth=3) # ValueError: x and y must have same first dimension, but have shapes (1383,) and (1729,)
axes[1].plot(date_test, test_absolute_error, color='r', label='Błąd bezwzględny (test)', linewidth=3)
axes[1].set_xlabel('Data', fontsize=15)
axes[1].set_ylabel('Błąd bezwzględny', fontsize=15)
axes[1].set_title('Błąd bezwzględny', fontsize=30)
axes[1].legend(loc='upper right', fontsize=20)
axes[1].grid()

# Wykres błędu względnego
axes[2].plot(date_train, train_relative_error, color='b', label='Błąd względny', linewidth=3)
axes[2].plot(date_test, test_relative_error, color='r', label='Błąd względny (test)', linewidth=3)
axes[2].set_xlabel('Data', fontsize=15)
axes[2].set_ylabel('Błąd względny', fontsize=15)
axes[2].set_title('Błąd względny', fontsize=30)
axes[2].legend(loc='upper right', fontsize=20)
axes[2].grid()

# Wyświetlenie wykresów
fig.show()

## Korelacje Pearsona między stacjami

In [ ]:
# Funkcja do liczenia korelacji Pearsona między dwiema stacjami
def correlation_between_stations(station_1, station_2, lag):
    station_1_id = stations.index(station_1.upper())
    station_2_id = stations.index(station_2.upper())
    correlation = lagged_dfs[lag][station_1_id]['B00020S'].corr(lagged_dfs[0][station_2_id]['B00020S'])
    return round(correlation, 3)

# Stacje
stations = ['GŁOGÓW', 'ŚCINAWA', 'MALCZYCE', 'BRZEG DOLNY', 'OŁAWA', 'BRZEG', 'RACIBÓRZ-MIEDONIA', 'KRZYŻANOWICE', 'OLZA', 'CHAŁUPKI']

# Grupowanie po dniach i stacjach
data_grouped = data.groupby(['Date', 'Station'])['B00020S'].mean().reset_index()

# Maksymalny lag
max_lag = 7

# Lista od 0 do max_lag
lags = range(max_lag + 1)

# Dwuwymiarowa lista zlagowanych dataframe'ów stacji
# Dostęp do wybranej stacji i lagu: lagged_dfs[lag][id_stacji np. 0 dla Głogowa]
start_date = '2008-01-08'
end_date = '2023-09-30'
lagged_dfs =[[data_grouped[(data_grouped['Date']
                .between(
                    pd.to_datetime(start_date) - pd.DateOffset(days=lag),
                    pd.to_datetime(end_date) - pd.DateOffset(days=lag)
                ))&(data_grouped['Station'] == station)].reset_index(drop=True) for station in stations]for lag in lags
             ]
# Lista stacji, dla których chcemy obliczyć korelację Pearsona względem Głogowa
stations_to_calculate_corr = ['ŚCINAWA', 'MALCZYCE']

# Wyświetlanie korelacji Pearsona
for station in stations_to_calculate_corr:
    print(f"\nWspółczynnik korelacji Pearsona liczony na podstawie poziomu wody w stacjach: {station.capitalize()} - Głogów")
    for lag in lags:
        print(f"Lag: {lag}, p = {correlation_between_stations(station, 'Głogów', lag)}")

In [4]:
train_data

,GŁOGÓW,ŚCINAWA,MALCZYCE,BRZEG DOLNY,OŁAWA,BRZEG,RACIBÓRZ-MIEDONIA,KRZYŻANOWICE,OLZA,CHAŁUPKI
0,226.270833,145.652778,145.340278,257.541667,163.298611,163.777778,160.517483,125.173611,142.243056,115.875000
1,231.680556,142.138889,121.152778,234.090278,164.753472,164.958333,154.666667,122.180556,141.000000,115.590278
2,239.347222,128.979167,127.034722,245.069444,159.663194,157.229167,163.202797,128.777778,145.375000,119.791667
3,237.090278,134.125000,128.604167,243.076389,161.340278,160.451389,154.868056,119.458333,138.465278,112.861111
4,230.763889,125.187500,99.229167,219.298611,166.434028,165.437500,148.361111,115.631944,136.250000,110.944444
...,...,...,...,...,...,...,...,...,...,...
1378,249.727273,132.888889,134.104167,330.187500,170.708333,173.506944,146.326389,96.715278,156.736111,103.486111
1379,251.034722,138.708333,124.875000,331.090909,169.555556,172.583333,145.187500,95.638889,156.111111,103.118056
1380,250.447552,132.833333,106.409722,329.166667,149.677083,170.534722,142.597222,93.583333,155.090909,100.527778
1381,239.564286,135.597222,113.888889,330.500000,150.645833,170.638889,142.180556,93.097222,154.611111,100.416667
